In [104]:
import pandas as pd
from google.cloud import datastore
import json

In [223]:
client = datastore.Client()
collection = 'suppliers'

query = client.query(kind=collection)
query.order = ['Name']

results = list(query.fetch())
supplier_count = len(results)


suppliers = []
for r in results:
    s = datastore.Entity(r)
    supplier = {}
    supplier['ABN'] = s.key.key.name
    
    suppliers.append(supplier)


In [224]:
df = pd.read_json(json.dumps(suppliers))

In [225]:
df.head()

,ABN
0,51137899346
1,95127484904
2,49800667911
3,34129384032
4,57070045117


In [236]:
contracts = []

def getContracts(ABN):
    #print(ABN)
    current_url = 'https://www.tenders.gov.au/?event=public.CN.searchDownload&download=true&atmtype=archived%2Cclosed%2Cpublished%2Cproposed&keywordtypesearch=AllWord&postcode=&valuefrom=&datestart=&keyword=&contractto=&suppliername=&valueto=&category=&cnid=&agencyuuid=&orderBy=Relevance&type=cnSearchEvent&portfoliouuid=&dateend=&publishfrom=&supplierabn=' + ABN + '&datetype=current&agencyrefid=&contractfrom=&atmid=&consultancy=&agencyStatus=&numagencystatus=%2D1&publishto=&multype=archived%2Cclosed%2Cpublished'
    
    current_ABN_contracts_df = (
        pd.read_csv(current_url, 
                skiprows=16, 
                delimiter='\t',
                parse_dates=['Contract Start Date', 'Contract End Date', 'Publish Date']
               )
    )
    
    #current_ABN_contracts_df['ABN'] = ABN
    
    if current_ABN_contracts_df.iloc[0].str.contains('no results', case=False, na=False)['CN ID'] == False:
        contracts.append(current_ABN_contracts_df)
        return True
    else:
        return None
        

In [238]:
from google.cloud import datastore

client = datastore.Client()
collection = 'supplier_contracts'


for abn in df['ABN']:
    
    print('Scraping contracts for ABN: {}'.format(abn))
    
    contracts = []
    
    if (getContracts(abn) != None):
    
        contracts[0] = contracts[0].drop(['Supplier Name', 'LastUpdated', 'Unnamed: 11'], axis=1)
        contracts[0]['CN ID'] = contracts[0]['CN ID'].map(lambda s: s.replace('="', '').replace('"', ''))
        contracts[0]['Agency'] = contracts[0]['Agency'].map(lambda s: s.replace('="', '').replace('"', ''))
        contracts[0]['Category'] = contracts[0]['Category'].map(lambda s: s.replace('="', '').replace('"', ''))
        contracts[0]['ATM ID'] = contracts[0]['ATM ID'].map(lambda s: s.replace('="', '').replace('"', ''))
        contracts[0].index = contracts[0]['CN ID']
        contracts[0] = contracts[0].drop('CN ID', axis=1)

        supplier_contracts = {}
        supplier_contracts['ABN'] = abn
        supplier_contracts['Contracts'] = contracts[0].to_json(orient='index')

        contract_details = json.loads(supplier_contracts['Contracts'])

        for contract_id in contract_details.keys():
            key = client.key(collection, contract_id)
            record = datastore.Entity(key=key)
            record['ABN'] = supplier_contracts['ABN']
            record['Contract ID'] = contract_id
            record['Agency'] = contract_details[contract_id].get('Agency')
            record['Category'] = contract_details[contract_id].get('Category')
            record['Start Date'] = contract_details[contract_id].get('Contract Start Date')
            record['End Date'] = contract_details[contract_id].get('Contract End Date')
            record['Title'] = contract_details[contract_id].get('Title')
            record['Value'] = contract_details[contract_id].get('Value (AUD)')
            record['ATM ID'] = contract_details[contract_id].get('ATM ID')

            client.put(record)
            print('Saved contract {} for ABN: {}'.format(record.key.name, abn))
        
    else:
        print('No contracts found')


Scraping contracts for ABN: 51137899346
No contracts found
Scraping contracts for ABN: 95127484904
Saved contract CN3370704 for ABN: 95127484904
Scraping contracts for ABN: 49800667911
Saved contract CN3433151 for ABN: 49800667911
Saved contract CN3433149 for ABN: 49800667911
Scraping contracts for ABN: 34129384032
Saved contract CN3345287 for ABN: 34129384032
Saved contract CN3442250 for ABN: 34129384032
Saved contract CN3432459 for ABN: 34129384032
Saved contract CN3421139 for ABN: 34129384032
Saved contract CN3432611 for ABN: 34129384032
Saved contract CN3374309 for ABN: 34129384032
Saved contract CN3445910 for ABN: 34129384032
Saved contract CN3446043 for ABN: 34129384032
Saved contract CN3442458 for ABN: 34129384032
Saved contract CN3432458 for ABN: 34129384032
Saved contract CN3370188 for ABN: 34129384032
Saved contract CN3387819 for ABN: 34129384032
Saved contract CN3438126 for ABN: 34129384032
Saved contract CN3442469 for ABN: 34129384032
Saved contract CN3442254 for ABN: 34129

Saved contract CN3447401 for ABN: 14007145637
Saved contract CN3448067 for ABN: 14007145637
Saved contract CN3433364 for ABN: 14007145637
Saved contract CN3404708 for ABN: 14007145637
Saved contract CN3409955 for ABN: 14007145637
Saved contract CN3433643 for ABN: 14007145637
Saved contract CN3346238-A1 for ABN: 14007145637
Saved contract CN3339116-A1 for ABN: 14007145637
Saved contract CN3285306-A2 for ABN: 14007145637
Saved contract CN3428986 for ABN: 14007145637
Saved contract CN3433695 for ABN: 14007145637
Saved contract CN3341276-A1 for ABN: 14007145637
Saved contract CN3442794 for ABN: 14007145637
Saved contract CN3445724 for ABN: 14007145637
Saved contract CN3437489 for ABN: 14007145637
Saved contract CN3433615 for ABN: 14007145637
Saved contract CN3447403 for ABN: 14007145637
Saved contract CN3321059-A2 for ABN: 14007145637
Saved contract CN3408240-A1 for ABN: 14007145637
Saved contract CN3433627 for ABN: 14007145637
Saved contract CN3448714 for ABN: 14007145637
Saved contract C

Saved contract CN3290087-A1 for ABN: 60103121464
Saved contract CN3373448-A1 for ABN: 60103121464
Saved contract CN3429322 for ABN: 60103121464
Saved contract CN3448594 for ABN: 60103121464
Saved contract CN3379487 for ABN: 60103121464
Saved contract CN3323736-A1 for ABN: 60103121464
Saved contract CN3332349-A2 for ABN: 60103121464
Saved contract CN3367861-A5 for ABN: 60103121464
Saved contract CN3439343 for ABN: 60103121464
Saved contract CN3295962-A2 for ABN: 60103121464
Saved contract CN3323730-A1 for ABN: 60103121464
Saved contract CN3349322-A1 for ABN: 60103121464
Saved contract CN3439270 for ABN: 60103121464
Saved contract CN3349307-A1 for ABN: 60103121464
Saved contract CN3437518 for ABN: 60103121464
Saved contract CN3430552 for ABN: 60103121464
Saved contract CN3447744 for ABN: 60103121464
Saved contract CN3443456 for ABN: 60103121464
Saved contract CN3367159-A1 for ABN: 60103121464
Saved contract CN3439747 for ABN: 60103121464
Saved contract CN3439745 for ABN: 60103121464
Save

Saved contract CN3435536 for ABN: 57139224436
Saved contract CN3342373-A2 for ABN: 57139224436
Saved contract CN3435537 for ABN: 57139224436
Saved contract CN3437903 for ABN: 57139224436
Saved contract CN3437857 for ABN: 57139224436
Saved contract CN3447414 for ABN: 57139224436
Saved contract CN3432239 for ABN: 57139224436
Saved contract CN3407129-A1 for ABN: 57139224436
Saved contract CN3447419 for ABN: 57139224436
Saved contract CN3442922 for ABN: 57139224436
Saved contract CN3436347 for ABN: 57139224436
Saved contract CN3423568-A1 for ABN: 57139224436
Saved contract CN3447412 for ABN: 57139224436
Saved contract CN3434578 for ABN: 57139224436
Saved contract CN3425401 for ABN: 57139224436
Saved contract CN3434081 for ABN: 57139224436
Saved contract CN3447416 for ABN: 57139224436
Saved contract CN3447413 for ABN: 57139224436
Saved contract CN3176452-A1 for ABN: 57139224436
Saved contract CN3421750 for ABN: 57139224436
Saved contract CN3408677 for ABN: 57139224436
Saved contract CN31765

Saved contract CN3404451-A1 for ABN: 43002724334
Saved contract CN3439392 for ABN: 43002724334
Saved contract CN3418029 for ABN: 43002724334
Saved contract CN3449744 for ABN: 43002724334
Saved contract CN3433050 for ABN: 43002724334
Saved contract CN3435629 for ABN: 43002724334
Saved contract CN3443344 for ABN: 43002724334
Saved contract CN3442773 for ABN: 43002724334
Saved contract CN3367557-A1 for ABN: 43002724334
Saved contract CN3329610-A2 for ABN: 43002724334
Saved contract CN3397197 for ABN: 43002724334
Saved contract CN3422765 for ABN: 43002724334
Saved contract CN3421973 for ABN: 43002724334
Saved contract CN3443728 for ABN: 43002724334
Saved contract CN3361672-A1 for ABN: 43002724334
Saved contract CN3446439 for ABN: 43002724334
Saved contract CN3412886-A1 for ABN: 43002724334
Saved contract CN3410112 for ABN: 43002724334
Saved contract CN3447377 for ABN: 43002724334
Saved contract CN3421972-A2 for ABN: 43002724334
Saved contract CN3375381 for ABN: 43002724334
Saved contract C

...

Saved contract CN3431789 for ABN: 71008626131
Saved contract CN3433156 for ABN: 71008626131
Saved contract CN3350892 for ABN: 71008626131
Saved contract CN3354115 for ABN: 71008626131
Saved contract CN3441108 for ABN: 71008626131
Saved contract CN3278453-A2 for ABN: 71008626131
Saved contract CN3443528 for ABN: 71008626131
Saved contract CN3441111 for ABN: 71008626131
Saved contract CN3421382 for ABN: 71008626131
Saved contract CN3312529-A2 for ABN: 71008626131
Saved contract CN3443631 for ABN: 71008626131
Saved contract CN3409488 for ABN: 71008626131
Saved contract CN3414907 for ABN: 71008626131
Saved contract CN3285273-A1 for ABN: 71008626131
Saved contract CN3411405 for ABN: 71008626131
Saved contract CN3435643 for ABN: 71008626131
Saved contract CN3389644-A1 for ABN: 71008626131
Saved contract CN3333579-A2 for ABN: 71008626131
Saved contract CN3443409 for ABN: 71008626131
Saved contract CN3435461 for ABN: 71008626131
Saved contract CN3443629 for ABN: 71008626131
Saved contract CN33

Saved contract CN3342016-A1 for ABN: 95121819305
Saved contract CN3433488 for ABN: 95121819305
Saved contract CN3297711-A1 for ABN: 95121819305
Saved contract CN3433750 for ABN: 95121819305
Saved contract CN3421584 for ABN: 95121819305
Saved contract CN3411388 for ABN: 95121819305
Saved contract CN3376816-A1 for ABN: 95121819305
Saved contract CN3443735 for ABN: 95121819305
Saved contract CN3412216-A1 for ABN: 95121819305
Saved contract CN3448500 for ABN: 95121819305
Saved contract CN3441735 for ABN: 95121819305
Saved contract CN3439960 for ABN: 95121819305
Saved contract CN3443602 for ABN: 95121819305
Saved contract CN3380751-A1 for ABN: 95121819305
Saved contract CN3440054 for ABN: 95121819305
Saved contract CN3416030 for ABN: 95121819305
Saved contract CN3443624 for ABN: 95121819305
Saved contract CN3385108 for ABN: 95121819305
Saved contract CN2081391-A1 for ABN: 95121819305
Saved contract CN3411799 for ABN: 95121819305
Saved contract CN3153152-A2 for ABN: 95121819305
Saved contrac

Saved contract CN3448990 for ABN: 95121819305
Saved contract CN3190622-A1 for ABN: 95121819305
Saved contract CN3442461 for ABN: 95121819305
Saved contract CN3443524 for ABN: 95121819305
Saved contract CN3448909 for ABN: 95121819305
Saved contract CN3315462-A1 for ABN: 95121819305
Saved contract CN3437025 for ABN: 95121819305
Saved contract CN3427424 for ABN: 95121819305
Saved contract CN3304822-A1 for ABN: 95121819305
Saved contract CN3439301 for ABN: 95121819305
Saved contract CN3438764 for ABN: 95121819305
Saved contract CN3370769-A2 for ABN: 95121819305
Saved contract CN3331705-A1 for ABN: 95121819305
Saved contract CN3414759-A1 for ABN: 95121819305
Saved contract CN3430560 for ABN: 95121819305
Saved contract CN3322347-A2 for ABN: 95121819305
Saved contract CN3278485-A2 for ABN: 95121819305
Saved contract CN3314802-A2 for ABN: 95121819305
Saved contract CN3447690 for ABN: 95121819305
Saved contract CN3302998-A1 for ABN: 95121819305
Saved contract CN3409438 for ABN: 95121819305
Save

Saved contract CN3399711-A1 for ABN: 13155397541
Saved contract CN3399713-A1 for ABN: 13155397541
Saved contract CN3385877-A1 for ABN: 13155397541
Saved contract CN3184302-A1 for ABN: 13155397541
Saved contract CN3125702-A2 for ABN: 13155397541
Saved contract CN3449029 for ABN: 13155397541
Saved contract CN3385648-A1 for ABN: 13155397541
Saved contract CN3408708-A1 for ABN: 13155397541
Saved contract CN3299931-A3 for ABN: 13155397541
Saved contract CN3418934-A1 for ABN: 13155397541
Saved contract CN3399709-A1 for ABN: 13155397541
Saved contract CN3403179-A1 for ABN: 13155397541
Saved contract CN3443669 for ABN: 13155397541
Saved contract CN2554091 for ABN: 13155397541
Saved contract CN3386845 for ABN: 13155397541
Saved contract CN3387103-A1 for ABN: 13155397541
Saved contract CN3381792-A1 for ABN: 13155397541
Saved contract CN3320271-A1 for ABN: 13155397541
Saved contract CN3413743 for ABN: 13155397541
Saved contract CN3384262-A1 for ABN: 13155397541
Saved contract CN3435616 for ABN: 1

Saved contract CN3430265 for ABN: 52119884945
Saved contract CN3347376-A2 for ABN: 52119884945
Saved contract CN3374288 for ABN: 52119884945
Saved contract CN3427153 for ABN: 52119884945
Saved contract CN3437799 for ABN: 52119884945
Saved contract CN3440045 for ABN: 52119884945
Saved contract CN3434985 for ABN: 52119884945
Saved contract CN3435005 for ABN: 52119884945
Saved contract CN3438134 for ABN: 52119884945
Saved contract CN3442078 for ABN: 52119884945
Saved contract CN3360057 for ABN: 52119884945
Saved contract CN3438138 for ABN: 52119884945
Saved contract CN3347476-A2 for ABN: 52119884945
Saved contract CN3437796 for ABN: 52119884945
Saved contract CN3416559 for ABN: 52119884945
Saved contract CN3378474 for ABN: 52119884945
Saved contract CN3442380 for ABN: 52119884945
Saved contract CN3424183 for ABN: 52119884945
Saved contract CN3396589 for ABN: 52119884945
Saved contract CN3377982-A1 for ABN: 52119884945
Saved contract CN3425741 for ABN: 52119884945
Saved contract CN3442413 